In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output
from random import uniform
import time
from random import randint
np.set_printoptions(precision=4)
import random
import time

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
data=pd.read_json("drive/My Drive/Colab Notebooks/Musical_Instruments_5.json", lines=True)

In [6]:
data.shape

(10261, 9)

In [7]:
print(data.head())

       reviewerID        asin  ... unixReviewTime   reviewTime
0  A2IBPI20UZIR0U  1384719342  ...     1393545600  02 28, 2014
1  A14VAT5EAX3D9S  1384719342  ...     1363392000  03 16, 2013
2  A195EZSQDW3E21  1384719342  ...     1377648000  08 28, 2013
3  A2C00NNG1ZQQG2  1384719342  ...     1392336000  02 14, 2014
4   A94QU4C90B1AX  1384719342  ...     1392940800  02 21, 2014

[5 rows x 9 columns]


In [8]:
data.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [0]:
text = data[["reviewText","summary"]]

In [10]:
text.head()

,reviewText,summary
0,"Not much to write about here, but it does exac...",good
1,The product does exactly as it should and is q...,Jake
2,The primary job of this device is to block the...,It Does The Job Well
3,Nice windscreen protects my MXL mic and preven...,GOOD WINDSCREEN FOR THE MONEY
4,This pop filter is great. It looks and perform...,No more pops when I record my vocals.


In [11]:
import re
for i in range(10261):
    text["reviewText"][i] = re.sub('[^a-zA-Z]',' ',text["reviewText"][i])
    text["summary"][i] = re.sub('[^a-zA-Z]',' ',text["summary"][i])
    clear_output(wait=True)
    print("complete :- ",i)

complete :-  10260


# Natural Lanuage
using NLP we have converted the text data(Review) to a numerical matrix to use in the KNN Algorithm

In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
textData =[]
for i in range(10261):
    review = text["reviewText"][i]+" "+text["summary"][i]
    review = re.sub('[^a-zA-Z]',' ',review)
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    review = [ps.stem(word) for word in review]
    review = ' '.join(review)
    textData.append(review)
    clear_output(wait=True)
    print("complete :- ",i)

complete :-  10260


In [0]:
 from sklearn.feature_extraction.text import CountVectorizer
 cv = CountVectorizer()
 X = cv.fit_transform(textData).toarray()
 X = pd.DataFrame(X)

In [15]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,13097,13098,13099,13100,13101,13102,13103,13104,13105,13106,13107,13108,13109,13110,13111,13112,13113,13114,13115,13116,13117,13118,13119,13120,13121,13122,13123,13124,13125,13126,13127,13128,13129,13130,13131,13132,13133,13134,13135,13136
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
X.shape

(10261, 13137)

In [17]:
data.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [0]:
data1 = data[["reviewerID","asin","reviewerName","helpful","overall","unixReviewTime","reviewTime"]]

In [20]:
data1.head()

,reviewerID,asin,reviewerName,helpful,overall,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",5,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",5,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",5,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",5,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",5,1392940800,"02 21, 2014"


In [21]:
data1.shape

(10261, 7)

In [22]:
len(set(data1["asin"]))

900

In [134]:
movie_features_df=data1.pivot_table(index='asin',columns='reviewerID',values='overall').fillna(0)
movie_features_df.head()

reviewerID,A00625243BI8W1SSZNLMD,A10044ECXDUVKS,A102MU6ZC9H1N6,A109JTUZXO61UY,A109ME7C09HM2M,A10APIDAZISWQF,A10B2J2IRQXBWA,A10E3QH2FQUBLF,A10FM4ILBIMJJ7,A10H2F00ZOT8S2,A10HYGDU2NITYQ,A10KH8EN77ZKWH,A10N243R7A5ZW3,A10NJEIG56RHN5,A10VG94SAKVSC0,A10ZSXTQA264C7,A110ZEDSNASVCO,A118PM0B1PGWDA,A11E4FWMN9BXJD,A11INIL2YFJ137,A120FZ2ESIMA63,A121QRWXZIO6UP,A126XEMCLHPBNZ,A127K5WGHNUUH3,A12ABV9NU02O29,A12DQZKRKTNF5E,A12N7TJQR2RB9W,A12O5B8XNKNBOL,A12P4A1OC41KUO,A12SSZIN555FTL,A12YXGXV4MATDS,A1365RYO0BLEMI,A136IQFGB01KQB,A136M3QYHUVN9A,A13798OPDBLDCO,A13A81NN0NRD1S,A13GMS7FWV3TQ0,A13IKQCJKFAP5S,A13KBLFF4IZF7H,A13NWJUMVNS6YZ,...,AWBZIK5JYWB5J,AWCJ12KBO5VII,AWIG50VOI5VUV,AWKVQQZKTRFAL,AWQQ1QHCECDJ3,AWV58YYFEAUL0,AWYE428W5MRQN,AWYXB9L41T82S,AX11NOUMV8G95,AX69H7INJKE76,AXABTEYS7A4A8,AXG9N4QFS4QYP,AXJ19189TLBLJ,AXMXE3RT660HQ,AXMYGK3WC8BPP,AXP7888CP2222,AXP9CF1UTFRSU,AXTOICJWZBAJ0,AXU9VX024GPSS,AXWB93VKVML6K,AXWEQHTXQWR7Q,AXWI0P2EGDEQT,AXX6BZDD8K4JL,AXXGP6UT41KAS,AXXYMIJBD0J9G,AYJJDQQ4EZ5V3,AYQ46BHSK99YV,AYQCAPXJ81XTN,AYTKUTAP0VA53,AZ0LJNEP2VRD1,AZ9KESC05F6RI,AZAYBFPLEDFL7,AZBUUKQLYKUCL,AZCP5P3BARLS5,AZE83O4F1IJPR,AZJPNK73JF3XP,AZMHABTPXVLG3,AZMIKIG4BB6BZ,AZPDO6FLSMLFP,AZVME8JMPD3F4
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1384719342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Y2UT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005ML71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000068NSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000068NTU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [149]:
movie_features_df.shape

(900, 1429)

##Lets select a index for finding nearest neibhour

In [150]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10)

648


In [0]:
Reviewer = list(movie_features_df.columns)
Instrument = list(movie_features_df.index)

In [155]:
Instrument[648]

'B00315D0E4'

##finding 10 nearest neighbour of reviewer index 845

In [156]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1},----- with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendations for B00315D0E4:

1: B003BGO2OK,----- with distance of 0.6271634510960847:
2: B001L8IKLG,----- with distance of 0.7792962277324336:
3: B0041T53KI,----- with distance of 0.7894620197333703:
4: B000T9PE9E,----- with distance of 0.8045203927763735:
5: B000WGJ71U,----- with distance of 0.8045203927763735:
6: B002HFC6P8,----- with distance of 0.8045203927763735:
7: B00CFMWBLC,----- with distance of 0.8101642962493689:
8: B0002D0MFI,----- with distance of 0.8116891057113227:
9: B000EEL8US,----- with distance of 0.822695035460993:


In [0]:
query_index = np.random.choice(movie_features_df.shape[0])
Name = name[People[query_index]]

In [0]:
name=dict()
for i in range(10261):
    if data['reviewerID'][i] not in name:
        name[data['reviewerID'][i]] = data['reviewerName'][i]
Reviewer = list(movie_features_df.index)
Instrument = list(movie_features_df.columns)

##Here two Function Define
###findInstrumentScore:- 
it find the Musical Instrument which is rated by the coustomer for which we are going to recommend the musical Instrument
### take_top_Rated_Instrument:-
It Extract top 10 rated Instrumet by that coustomer for which we have to recommend musical Instrument

In [0]:
Rated ={1:[], 2:[], 3:[], 4:[], 5:[]}
def findInstrumentScore(X):
    print(Rated)
    for i in range(10261):
        if data1['reviewerName'][i] == X :
            Rated[round(data1['overall'][i])].append(data1['asin'][i])
    print(Rated)
def take_top_Rated_Instrument():
    Arr=[]
    for i in range(5,1,-1):
        Arr = list(set(Arr))
        if len(Arr) < 10:
            Arr+=(Rated[i])
        
    return random.choices(Arr, k=10)
        

## KNN function:-
It take the top 10 Instrument and find the their closest neighbour with distance using KNN Unsupervise Learning

In [0]:
Neighbour=dict()
def KNN(INDEX):
    for query_index in INDEX:
        distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)
        for i in range(0, len(distances.flatten())):
            if i == 0:
                Neighbour[movie_features_df.index[query_index]]=[]
            else:
                Neighbour[movie_features_df.index[query_index]].append([movie_features_df.index[indices.flatten()[i]],distances.flatten()[i]])
    return Neighbour

###Here we choose a Random Customer and predict Musical Instument Using KNN


In [222]:
query_index = np.random.choice(movie_features_df.shape[0])
Name = name[People[query_index]]

findInstrumentScore(Name)
Arr=take_top_Rated_Instrument()
INDEX = set()
for i in Arr:
    INDEX.add(Instrument.index(i))
print(*INDEX)
RESULT = KNN(INDEX)
AA=[]
for i in RESULT:
    AA+=RESULT[i]
AA.sort(key = lambda x: x[1])
for i in range(len(AA)):
    print('Costomer {0} ----having closest neighbour {1}   with distance   {2}'.format(Name,AA[i][0],AA[i][1]))

{1: ['B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY', 'B005BSOVRY'], 2: [], 3: [], 4: ['B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6', 'B008AOH1O6'], 5: ['B0002DVBJY', 'B0002E1G5C', 'B0002F6RSM', 'B0002IHP0S', 'B000RW2DUQ', 'B000SJJCX4', 'B005CX4GLE', 'B0002DVBJY', 'B0002E1G5C', 'B0002F6RSM', 'B0002IHP0S', 'B000RW2DUQ', 'B000SJJCX4', 'B005CX4GLE', 'B0002DVBJY', 'B0002E1G5C', 'B0002F6RSM', 'B0002IHP0S', 'B000RW2DUQ', 'B000SJJCX4', 'B005CX4GLE', 'B0002DVBJY', 'B0002E1G5C', 'B0002F6RSM', 'B0002IHP0S', 'B000RW2DUQ', 'B000SJJCX4', 'B005CX4GLE', 'B0002DVBJY', 'B0002E1G5C', 'B0002F6RSM', 'B0002IHP0S', 'B000RW2DUQ', 'B000SJJCX4', 'B005CX4GLE', 'B0002DVBJY', 'B0002E1G5C', 'B0002F6RSM', 'B0002IHP0S', 'B000RW2DUQ', 'B000SJJCX4', 'B005